# Accessing OpenAI Like a Developer

- 🤝 Breakout Room #1:
  1. Getting Started
  2. Setting Environment Variables
  3. Using the OpenAI Python Library
  4. Prompt Engineering Principles
  5. Testing Your Prompt

# How AIM Does Assignments

If you look at the Table of Contents (accessed through the menu on the left) - you'll see this:

![image](https://i.imgur.com/I8iDTUO.png)

Or this if you're in Colab:

![image](https://i.imgur.com/0rHA1yF.png)

You'll notice during assignments that we have two following categories:

1. ❓ - Questions. These will involve...answering questions!
2. 🏗️ - Activities. These will involve writing code, or modifying text.

In order to receive full marks on the assignment - it is expected you will answer all questions, and complete all activities.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [1]:
!pip install openai -q

## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [3]:
from openai import OpenAI

openai_client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the `gpt-3.5-turbo` model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-3.5-turbo` model as stated above.

Let's look at an example!



In [11]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : "Who is Mazinger Z?"}]
)

Let's look at the response object.

In [12]:
response

ChatCompletion(id='chatcmpl-9ZiTBrcLwChA0O2f6IrsBPsxaaeo5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Mazinger Z is a fictional giant robot manga and anime series created by Go Nagai. The story follows the adventures of scientist Juzo Kabuto and his giant robot Mazinger Z as they battle against evil forces threatening humanity. Mazinger Z is one of the first super robot anime series and is considered a classic in the genre.', role='assistant', function_call=None, tool_calls=None))], created=1718299449, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=68, prompt_tokens=13, total_tokens=81))

>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### Helper Functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [13]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

###Refactoring the Prompt Helpers
A bit of refactoring just for fun as I start to get a hang on this notebooks. The goal is to reduce the amount of functions and duplication of code lines.

In [14]:
def create_prompt(role: str, message: str) -> dict:
    return {"role": role, "content": message}

### Testing Helper Functions

Let's see how we can use these to help us!

In [15]:
YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Hello! I'm just a computer program, so I don't have feelings in the way that humans do. How can I assist you today?

### Testing my refactored version of the code
Let's see if it works.

In [16]:
YOUR_PROMPT = "What is code refactoring?"
messages_list = [create_prompt("user", YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Code refactoring is the process of restructuring existing computer code without changing its external behavior. The main goal of code refactoring is to improve the code's readability, efficiency, and maintainability without altering its functionality. This is typically done to make the code easier to understand, modify, and maintain in the future. Refactored code is usually cleaner, more organized, and easier to work with than the original code.

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [17]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself using PG-13 language."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

I don't give a damn about the shape of the ice! Just gimme something cold to eat before I pass out from hunger!

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [18]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

Oh, I love both! Crushed ice is so refreshing and perfect for blending into slushy drinks, while cubed ice is great for keeping my beverages cold without watering them down too quickly. It's like having the best of both worlds! Today is such a great day, isn't it? What can I help you with?

With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [19]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I accidentally spilled my stimple of coffee on my brand new falbean rug.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [20]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I needed to tighten the loose screw, so I grabbed my trusty stimple and falbean tool to get the job done quickly and efficiently.

As you can see, leveraging the `assistant` role makes for a stimple experience!

### 🏗️ Activity #1:

Use few-shop prompting to build a movie-review sentiment clasifier!

A few examples:

INPUT: "I hated the hulk!"
OUTPUT: "{"sentiment" : "negative"}

INPUT: "I loved The Marvels!"
OUTPUT: "{sentiment" : "positive"}

In [24]:
### YOUR CODE HERE
list_of_movie_review_prompts = [
    create_prompt("user", "Study the following samples of movie reviews and their correspondent sentiment:"),
    create_prompt("assistant", "INPUT: 'I hated The Hulk!' OUTPUT: 'sentiment: Rotten Tomatoes'"),
    create_prompt("assistant", "INPUT: 'I loved Bridesmaids!' OUTPUT: 'sentiment: Fresh Tomatoes'"),
    create_prompt("assistant", "INPUT: 'I Batman 3 sucked!' OUTPUT: 'sentiment: Rotten Tomatoes'"),
    create_prompt("assistant", "INPUT: 'The Godfather is the best movie of all times!' OUTPUT: 'sentiment: Fresh Tomatoes'"),
    create_prompt("assistant", "INPUT: 'Speed 2 was a waste of money and time!' OUTPUT: 'sentiment: Rotten Tomatoes'"),
    create_prompt("user", "Analyze this movie review and provide a sentiment score. Use 'Fresh Tomatoes' for positive sentiments and 'Rotten Tomatoes' for negative ones: 'Jaws is a great suspense movie!'"),
]

sentiment_analysis_response = get_response(openai_client, list_of_movie_review_prompts)

pretty_print(sentiment_analysis_response)

sentiment: Fresh Tomatoes

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

> NOTE: With improvements to `gpt-3.5-turbo`, this example might actually result in the correct response some percentage of the time!

In [25]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects. If he flies and takes a bus, the total travel time will be 5 hours (3 hours for the flight + 2 hours for the bus), which means he will arrive at his destination at 6PM local time. If he takes the teleporter and then a bus, the total travel time will be 1 hour (0 hours for the teleporter + 1 hour for the bus), which means he will arrive at his destination at 2PM local time. This option would allow him to arrive well before 7PM EDT. Therefore, Billy should choose the teleporter option to ensure he gets home before 7PM EDT.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [26]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects.

If Billy chooses to fly and then take a bus, it will take him a total of 5 hours to get home. If he leaves at 1PM local time, he will arrive home at around 6PM local time, which is before 7PM EDT.

If Billy chooses to take the teleporter and then a bus, it will only take him a total of 1 hour to get home. If he leaves at 1PM local time, he will arrive home at around 2PM local time, which is well before 7PM EDT.

Therefore, Billy should choose to take the teleporter and then a bus in order to get home before 7PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

## 3. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

### 🏗️ Activity #2:

There are two subtasks in this activity:

1. Write a `system_template` that leverages 2-3 of the principles from [this paper](https://arxiv.org/pdf/2312.16171v1.pdf)

2. Modify the `user_template` to improve the quality of the LLM's responses.

> NOTE: PLEASE DO NOT MODIFY THE `{input}` in the `user_template`.

In [30]:
system_template = """
You are an expert assistant focused on providing detailed, step-by-step guidance for completing complex tasks. Always ensure your responses are clear, concise, and contextually relevant to the user's input.
"""

In [31]:
user_template = """{input}
Considering the following context: {input}, provide a comprehensive and accurate response, breaking down the information into actionable steps where applicable.
"""

Messing around with the Response Template to check how it works. I am printing the response here to see if the requested structure is in place.

In [32]:
response_template = """
When providing an answer, follow this structure:

### Summary
Provide a brief summary of the key points in 2-3 sentences.

### Instructions
1. Render a numbered list with specific steps or actions.
2. Each step should be clear and concise.

### Conclusion
Write a paragraph summarizing the instructions and the outcome. This should wrap up the response and provide any final thoughts.

Ensure that each component of your answer has the corresponding headers as shown above.
"""

list_of_prompts = [
    user_prompt(response_template + "\n\nQuery: How can I improve my public speaking skills?")
]

structured_response = get_response(openai_client, list_of_prompts)
pretty_print(structured_response)

### Summary
Improving public speaking skills involves practice, preparation, and confidence-building techniques.

### Instructions
1. **Practice Regularly**: Practice speaking in front of a mirror, recording yourself, or joining a public speaking group.
2. **Prepare Thoroughly**: Research your topic, create an outline, and rehearse your speech multiple times.
3. **Work on Body Language**: Maintain eye contact, use gestures to emphasize points, and practice good posture.
4. **Engage with Your Audience**: Connect with your audience by asking questions, telling stories, and being relatable.
5. **Manage Nervousness**: Practice deep breathing, visualization techniques, and positive affirmations to calm nerves.

### Conclusion
Improving public speaking skills requires dedication and practice. By regularly practicing, preparing thoroughly, working on body language, engaging with the audience, and managing nervousness, you can become a confident and effective public speaker. Remember, the more you speak in public, the more comfortable and skilled you will become.

## 4. Testing Your Prompt

Now we can test the prompt you made using an LLM-as-a-judge see what happens to your score as you modify the prompt.

In [33]:
query = "How can I improve my public speaking skills?"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

Improving public speaking skills involves practice, preparation, and continuous learning. Here are some actionable steps to enhance your public speaking abilities:

1. **Practice Regularly**: Practice speaking in front of a mirror, record yourself, or speak in front of friends/family to build confidence and refine your delivery.

2. **Join a Public Speaking Group**: Consider joining a Toastmasters club or a similar group where you can practice regularly, receive feedback, and learn from others.

3. **Prepare Thoroughly**: Research your topic well, outline key points, and structure your speech to ensure clarity and coherence.

4. **Work on Body Language**: Practice good posture, gestures, and eye contact to engage your audience and appear confident.

5. **Manage Nervousness**: Techniques such as deep breathing, visualization, and positive self-talk can help calm nerves before speaking publicly.

6. **Engage the Audience**: Use storytelling, humor, or interactive elements to keep the audience interested and involved.

7. **Work on Vocal Variety**: Practice varying your tone, volume, and pace to maintain audience interest and emphasize key points.

8. **Seek Feedback**: Ask for constructive feedback from peers, mentors, or audience members to identify areas for improvement.

9. **Attend Workshops or Courses**: Consider enrolling in public speaking workshops or courses to learn from experts and expand your skill set.

10. **Watch and Learn from Experienced Speakers**: Observe and study effective speakers to understand techniques they use and adapt them to your style.

Remember, improvement takes time and dedication. Consistent practice and a willingness to learn from both successes and setbacks will help you progress in your public speaking journey.

In [34]:
pretty_print(evaluator_response)

{
"clarity" : 9,
"faithfulness" : 10,
"correctness" : 9
}

   

#### ❓Question #1:

How did your prompting strategies change the evaluation scores? What does this tell you/what did you learn?

> The main learnin for me was that prompt engineering is kind of an art more than a technique. You need to know the techniques and patterns for sure, but since the output is not deterministic, you always get a different answer. So being able to tweak and test your prompts regularly will yield better results and will show you what works and what doesn't. For me, the most impressive discovery was the RESPONSE_TEMPLATE. It has an immediate impact on the quality of the outputs and makes it more useful. I got to play with different ways to structure the response to tailor it to my AIE3 notes taking. Every time I run through this Notebook I learn something new.